In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable


In [2]:
manualSeed = random.randint(1, 10000)
#manualSeed = 4532
print("Random Seed: ", manualSeed)
random.seed(manualSeed)

torch.manual_seed(manualSeed)

torch.cuda.manual_seed_all(manualSeed)

cudnn.benchmark = True


Random Seed:  6832


In [3]:
dataroot = "./geom_dataset"
imageSize = 64
# folder dataset
dataset = dset.ImageFolder(root=dataroot,
                               transform=transforms.Compose([
                                   transforms.Scale(imageSize),
                                   transforms.CenterCrop(imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))

In [4]:
batchSize= 32
workers = 2
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
                                         shuffle=True, num_workers=int(workers))



In [5]:
print('num of batches:', dataloader.__len__())
print('num Of Images:', dataloader.__len__()*batchSize)

num of batches: 169
num Of Images: 5408


In [6]:
ngpu = int(2)
nz = int(100)
ngf = int(64)
ndf = int(64)
nc = 3

In [7]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [8]:
class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output


In [9]:
netG = _netG(ngpu)
netG.apply(weights_init)

#netG.load_state_dict(torch.load(savednetG))
print(netG)


_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh ()
  )
)


In [10]:
class _netD(nn.Module):
    def __init__(self, ngpu):
        super(_netD, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


In [11]:
netD = _netD(ngpu)
netD.apply(weights_init)

#netD.load_state_dict(torch.load(savednetD))
print(netD)


_netD (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid ()
  )
)


In [12]:
# Binary Cross Entropy between the target (p, 1-p) and the output (q,1-q) distributions
criterion = nn.BCELoss()

In [13]:
noise = torch.FloatTensor(batchSize, nz, 1, 1)
noise = noise.cuda()
noise.size()

torch.Size([32, 100, 1, 1])

In [14]:
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
fixed_noise = Variable(fixed_noise)
fixed_noise = fixed_noise.cuda()
fixed_noise.size()

torch.Size([32, 100, 1, 1])

In [15]:
real_label = 1
fake_label = 0

In [16]:
netD.cuda()
netG.cuda()
criterion.cuda()

BCELoss (
)

In [17]:
beta1 =0.5
lr=0.0002
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr, betas=(beta1, 0.999))

#optimizerD = optim.SGD(netD.parameters(), lr=0.001, momentum=0.9)
#optimizerG = optim.SGD(netG.parameters(), lr=0.001, momentum=0.9)

In [18]:
dataiter = iter(dataloader)
data= dataiter.next()


In [19]:
print(data[0].size(),data[1].size())

torch.Size([32, 3, 64, 64]) torch.Size([32])


In [20]:
netD.zero_grad()
real_cpu, _ = data
real_cpu = real_cpu.cuda()
real_cpu.size()

torch.Size([32, 3, 64, 64])

In [21]:
batch_size = real_cpu.size(0)
batch_size

32

In [22]:
input = torch.FloatTensor(batchSize, 3, imageSize, imageSize)
input = input.cuda()
input.resize_as_(real_cpu).copy_(real_cpu)
input.size()

torch.Size([32, 3, 64, 64])

In [23]:
label = torch.FloatTensor(batchSize)
label = label.cuda()
label.resize_(batch_size).fill_(real_label)
label.size()

torch.Size([32])

In [24]:
inputv = Variable(input)
real_input_tensor=input

labelv = Variable(label)

print(inputv.size(),labelv.size())

torch.Size([32, 3, 64, 64]) torch.Size([32])


In [25]:
output = netD(inputv)
output.size()       

torch.Size([32])

In [26]:
#print(output, labelv)

In [27]:
# We want the output of netD(real images) to be all ones
errD_real = criterion(output, labelv)
errD_real

Variable containing:
 0.5564
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [28]:
# We want D_x to be close to 1
D_x = output.data.mean()
print(D_x)

0.624069750309


In [29]:
errD_real.backward()

In [30]:
# train with fake
noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
noisev = Variable(noise)
noisev.size()

torch.Size([32, 100, 1, 1])

In [31]:
#  Generate fake images from noise
fake = netG(noisev)
fake.size()

torch.Size([32, 3, 64, 64])

In [32]:
labelv = Variable(label.fill_(fake_label))
labelv.size()

torch.Size([32])

In [33]:
output = netD(fake.detach())
output.size()

torch.Size([32])

In [34]:
#We want the output of netD(fake images) to be all zeros
#print(output, labelv)

In [35]:
errD_fake = criterion(output, labelv)
errD_fake

Variable containing:
 1.4522
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [36]:
errD_fake.backward()

In [37]:
# We want D_G_z1 to be close to 0
D_G_z1 = output.data.mean()
D_G_z1

0.6974189877510071

In [38]:
# The combined error:
errD = errD_real + errD_fake
errD

Variable containing:
 2.0087
[torch.cuda.FloatTensor of size 1 (GPU 0)]

 Update D network: maximize log(D(x)) + log(1 - D(G(z)))
 We want D to be as good as possible
 


In [39]:
optimizerD.step()


Evaluation to see if optimizing netD helped

In [40]:
real_label_vec=Variable(label.resize_(batch_size).fill_(real_label)).cuda()
new_output_real = netD(Variable(real_input_tensor))
new_D_x = new_output_real.data.mean()
print('new D_x', new_D_x)
new_errD_real = criterion(new_output_real, real_label_vec)
print('new errD_real', new_errD_real)

new D_x 0.955706059933
new errD_real Variable containing:
1.00000e-02 *
  4.6155
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [41]:
#print(new_output_real,real_labelvec)

In [42]:
fake_label_vec=Variable(label.resize_(batch_size).fill_(fake_label)).cuda()
new_output_fake = netD(netG(Variable(noise)))
new_D_G_z1 = new_output_fake.data.mean()
print('new D_G_z1: ', new_D_G_z1)
# We want D_G_z1 to be as close to zero as possible
new_errD_fake = criterion(new_output_fake, fake_label_vec)
print('new_errD_fake: ',new_errD_fake)

new D_G_z1:  0.00169973191805
new_errD_fake:  Variable containing:
1.00000e-03 *
  1.7029
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [43]:
new_errD = new_errD_real + new_errD_fake
new_errD

Variable containing:
1.00000e-02 *
  4.7858
[torch.cuda.FloatTensor of size 1 (GPU 0)]

(2) Update G network: maximize log(D(G(z)))

In [44]:
netG.zero_grad()

In [45]:
labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
labelv.size()

torch.Size([32])

In [46]:
output = netD(fake)
output.size()

torch.Size([32])

In [47]:
#print(output, labelv)

In [48]:
# We want this to be as small as possible
errG = criterion(output, labelv)
errG

Variable containing:
 6.8178
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [49]:
errG.backward()

In [50]:
# We want this to be as close to one as possible (we want good fake data)
D_G_z2 = output.data.mean()
D_G_z2

0.0016997314523905516

In [51]:
optimizerG.step()

Evalutation to see if netG optimizer helped

In [52]:
new_output = netD(netG(noisev))
# we want these to be all ones
new_errG = criterion(new_output, labelv)
# We want the error to be as small as possible
new_errG

Variable containing:
1.00000e-02 *
  3.3706
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [53]:
# We want this to be as close to one as possible (we want good fake data)
new_D_G_z2 = new_output.data.mean()
new_D_G_z2

0.9670742154121399

In [54]:
# we want these to be all ones
#print(new_output, labelv)

In [55]:
niter = 40
epoch=1
i=1
print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(dataloader),
                 errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))

[1/40][1/169] Loss_D: 2.0087 Loss_G: 6.8178 D(x): 0.6241 D(G(z)): 0.6974 / 0.0017


In [56]:
outf="./dcgan_results"
vutils.save_image(real_cpu,
                    '%s/real_samples.png' % outf,
                    normalize=True)
            

In [57]:
epoch=1
fake = netG(fixed_noise)
vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (outf, epoch),
                    normalize=True)

Let us put everything into two loops (epoch, minibatches)

In [58]:
use_cuda=True
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        if use_cuda:
            real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(dataloader),
                 errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (outf, epoch),
                    normalize=True)


[0/40][0/169] Loss_D: 0.5111 Loss_G: 6.9060 D(x): 0.8706 D(G(z)): 0.2675 / 0.0013
[0/40][1/169] Loss_D: 0.2027 Loss_G: 6.6379 D(x): 0.9434 D(G(z)): 0.1252 / 0.0016
[0/40][2/169] Loss_D: 0.1913 Loss_G: 6.6550 D(x): 0.9540 D(G(z)): 0.1236 / 0.0017
[0/40][3/169] Loss_D: 0.2328 Loss_G: 7.3384 D(x): 0.9583 D(G(z)): 0.1535 / 0.0008
[0/40][4/169] Loss_D: 0.1334 Loss_G: 7.3117 D(x): 0.9681 D(G(z)): 0.0913 / 0.0008
[0/40][5/169] Loss_D: 0.1910 Loss_G: 6.3881 D(x): 0.9168 D(G(z)): 0.0875 / 0.0020
[0/40][6/169] Loss_D: 0.4009 Loss_G: 9.7593 D(x): 0.9814 D(G(z)): 0.2870 / 0.0001
[0/40][7/169] Loss_D: 0.1153 Loss_G: 7.8946 D(x): 0.9241 D(G(z)): 0.0248 / 0.0005
[0/40][8/169] Loss_D: 0.1410 Loss_G: 7.6646 D(x): 0.9693 D(G(z)): 0.0976 / 0.0006
[0/40][9/169] Loss_D: 0.1648 Loss_G: 9.2615 D(x): 0.9697 D(G(z)): 0.1109 / 0.0001
[0/40][10/169] Loss_D: 0.1577 Loss_G: 6.6399 D(x): 0.8905 D(G(z)): 0.0155 / 0.0018
[0/40][11/169] Loss_D: 0.2718 Loss_G: 11.0502 D(x): 0.9657 D(G(z)): 0.1921 / 0.0000
[0/40][12/169

[0/40][100/169] Loss_D: 0.1362 Loss_G: 4.4635 D(x): 0.9195 D(G(z)): 0.0381 / 0.0148
[0/40][101/169] Loss_D: 0.1509 Loss_G: 4.8118 D(x): 0.9419 D(G(z)): 0.0818 / 0.0104
[0/40][102/169] Loss_D: 0.1847 Loss_G: 4.5738 D(x): 0.9054 D(G(z)): 0.0503 / 0.0136
[0/40][103/169] Loss_D: 0.1204 Loss_G: 4.6654 D(x): 0.9466 D(G(z)): 0.0554 / 0.0119
[0/40][104/169] Loss_D: 0.1231 Loss_G: 4.6067 D(x): 0.9339 D(G(z)): 0.0478 / 0.0121
[0/40][105/169] Loss_D: 0.1054 Loss_G: 5.2357 D(x): 0.9715 D(G(z)): 0.0705 / 0.0061
[0/40][106/169] Loss_D: 0.1812 Loss_G: 4.6570 D(x): 0.8997 D(G(z)): 0.0402 / 0.0119
[0/40][107/169] Loss_D: 0.0691 Loss_G: 4.9552 D(x): 0.9850 D(G(z)): 0.0518 / 0.0084
[0/40][108/169] Loss_D: 0.1098 Loss_G: 4.8261 D(x): 0.9461 D(G(z)): 0.0444 / 0.0092
[0/40][109/169] Loss_D: 0.0883 Loss_G: 5.0212 D(x): 0.9724 D(G(z)): 0.0573 / 0.0076
[0/40][110/169] Loss_D: 0.0822 Loss_G: 5.2908 D(x): 0.9661 D(G(z)): 0.0413 / 0.0060
[0/40][111/169] Loss_D: 0.1468 Loss_G: 4.1469 D(x): 0.9034 D(G(z)): 0.0350 /

[1/40][30/169] Loss_D: 0.1303 Loss_G: 4.4150 D(x): 0.9133 D(G(z)): 0.0281 / 0.0168
[1/40][31/169] Loss_D: 0.1830 Loss_G: 5.6782 D(x): 0.9473 D(G(z)): 0.0931 / 0.0054
[1/40][32/169] Loss_D: 0.1061 Loss_G: 4.9882 D(x): 0.9389 D(G(z)): 0.0370 / 0.0114
[1/40][33/169] Loss_D: 0.0944 Loss_G: 4.8748 D(x): 0.9580 D(G(z)): 0.0458 / 0.0112
[1/40][34/169] Loss_D: 0.0968 Loss_G: 5.9465 D(x): 0.9828 D(G(z)): 0.0750 / 0.0041
[1/40][35/169] Loss_D: 0.1080 Loss_G: 5.0140 D(x): 0.9228 D(G(z)): 0.0201 / 0.0111
[1/40][36/169] Loss_D: 0.1686 Loss_G: 3.4212 D(x): 0.9152 D(G(z)): 0.0439 / 0.0385
[1/40][37/169] Loss_D: 0.1943 Loss_G: 5.9297 D(x): 0.9518 D(G(z)): 0.1230 / 0.0041
[1/40][38/169] Loss_D: 0.1355 Loss_G: 5.0964 D(x): 0.9217 D(G(z)): 0.0222 / 0.0097
[1/40][39/169] Loss_D: 0.0515 Loss_G: 4.8510 D(x): 0.9864 D(G(z)): 0.0366 / 0.0119
[1/40][40/169] Loss_D: 0.0565 Loss_G: 5.7247 D(x): 0.9904 D(G(z)): 0.0430 / 0.0047
[1/40][41/169] Loss_D: 0.0717 Loss_G: 4.9008 D(x): 0.9602 D(G(z)): 0.0275 / 0.0104
[1/4

[1/40][131/169] Loss_D: 0.0908 Loss_G: 4.6238 D(x): 0.9421 D(G(z)): 0.0284 / 0.0145
[1/40][132/169] Loss_D: 0.1046 Loss_G: 5.2570 D(x): 0.9794 D(G(z)): 0.0778 / 0.0076
[1/40][133/169] Loss_D: 0.1050 Loss_G: 5.2218 D(x): 0.9519 D(G(z)): 0.0471 / 0.0082
[1/40][134/169] Loss_D: 0.1020 Loss_G: 5.0560 D(x): 0.9588 D(G(z)): 0.0489 / 0.0132
[1/40][135/169] Loss_D: 0.1035 Loss_G: 4.4946 D(x): 0.9448 D(G(z)): 0.0435 / 0.0182
[1/40][136/169] Loss_D: 0.1846 Loss_G: 5.4556 D(x): 0.9509 D(G(z)): 0.1056 / 0.0065
[1/40][137/169] Loss_D: 0.0617 Loss_G: 5.6771 D(x): 0.9551 D(G(z)): 0.0145 / 0.0068
[1/40][138/169] Loss_D: 0.0968 Loss_G: 4.4473 D(x): 0.9481 D(G(z)): 0.0350 / 0.0181
[1/40][139/169] Loss_D: 0.0846 Loss_G: 4.7587 D(x): 0.9802 D(G(z)): 0.0599 / 0.0124
[1/40][140/169] Loss_D: 0.0831 Loss_G: 5.0779 D(x): 0.9665 D(G(z)): 0.0460 / 0.0094
[1/40][141/169] Loss_D: 0.0671 Loss_G: 4.7735 D(x): 0.9609 D(G(z)): 0.0242 / 0.0104
[1/40][142/169] Loss_D: 0.0775 Loss_G: 4.6716 D(x): 0.9675 D(G(z)): 0.0419 /

[2/40][64/169] Loss_D: 0.1567 Loss_G: 7.4737 D(x): 0.9876 D(G(z)): 0.1218 / 0.0010
[2/40][65/169] Loss_D: 0.2866 Loss_G: 4.7637 D(x): 0.8198 D(G(z)): 0.0049 / 0.0213
[2/40][66/169] Loss_D: 0.0986 Loss_G: 4.8658 D(x): 0.9952 D(G(z)): 0.0872 / 0.0110
[2/40][67/169] Loss_D: 0.0702 Loss_G: 6.2897 D(x): 0.9929 D(G(z)): 0.0585 / 0.0030
[2/40][68/169] Loss_D: 0.0334 Loss_G: 6.2222 D(x): 0.9850 D(G(z)): 0.0177 / 0.0030
[2/40][69/169] Loss_D: 0.1922 Loss_G: 3.6766 D(x): 0.8801 D(G(z)): 0.0127 / 0.0425
[2/40][70/169] Loss_D: 0.2344 Loss_G: 8.2672 D(x): 0.9957 D(G(z)): 0.1833 / 0.0005
[2/40][71/169] Loss_D: 0.1549 Loss_G: 6.7451 D(x): 0.8675 D(G(z)): 0.0017 / 0.0026
[2/40][72/169] Loss_D: 0.0209 Loss_G: 5.5851 D(x): 0.9840 D(G(z)): 0.0045 / 0.0070
[2/40][73/169] Loss_D: 0.0305 Loss_G: 4.9653 D(x): 0.9979 D(G(z)): 0.0274 / 0.0124
[2/40][74/169] Loss_D: 0.0588 Loss_G: 5.1520 D(x): 0.9944 D(G(z)): 0.0509 / 0.0079
[2/40][75/169] Loss_D: 0.0344 Loss_G: 5.7172 D(x): 0.9821 D(G(z)): 0.0159 / 0.0050
[2/4

...

[37/40][111/169] Loss_D: 0.0207 Loss_G: 6.0937 D(x): 0.9830 D(G(z)): 0.0033 / 0.0049
[37/40][112/169] Loss_D: 0.0166 Loss_G: 5.4863 D(x): 0.9989 D(G(z)): 0.0152 / 0.0096
[37/40][113/169] Loss_D: 0.0164 Loss_G: 6.0400 D(x): 0.9977 D(G(z)): 0.0136 / 0.0063
[37/40][114/169] Loss_D: 0.0227 Loss_G: 5.5830 D(x): 0.9831 D(G(z)): 0.0054 / 0.0055
[37/40][115/169] Loss_D: 0.0067 Loss_G: 6.2794 D(x): 0.9972 D(G(z)): 0.0038 / 0.0036
[37/40][116/169] Loss_D: 0.0114 Loss_G: 5.6226 D(x): 0.9968 D(G(z)): 0.0082 / 0.0062
[37/40][117/169] Loss_D: 0.0139 Loss_G: 5.9463 D(x): 0.9912 D(G(z)): 0.0049 / 0.0049
[37/40][118/169] Loss_D: 0.0076 Loss_G: 6.4936 D(x): 0.9969 D(G(z)): 0.0044 / 0.0034
[37/40][119/169] Loss_D: 0.0070 Loss_G: 6.9490 D(x): 0.9951 D(G(z)): 0.0021 / 0.0021
[37/40][120/169] Loss_D: 0.0075 Loss_G: 6.6227 D(x): 0.9958 D(G(z)): 0.0033 / 0.0032
[37/40][121/169] Loss_D: 0.0066 Loss_G: 7.0752 D(x): 0.9953 D(G(z)): 0.0019 / 0.0020
[37/40][122/169] Loss_D: 0.0134 Loss_G: 5.8323 D(x): 0.9954 D(G(z

[38/40][40/169] Loss_D: 0.0260 Loss_G: 5.6962 D(x): 0.9900 D(G(z)): 0.0156 / 0.0066
[38/40][41/169] Loss_D: 0.0106 Loss_G: 6.7018 D(x): 0.9933 D(G(z)): 0.0037 / 0.0029
[38/40][42/169] Loss_D: 0.0345 Loss_G: 5.7240 D(x): 0.9685 D(G(z)): 0.0023 / 0.0066
[38/40][43/169] Loss_D: 0.0213 Loss_G: 5.3235 D(x): 0.9974 D(G(z)): 0.0181 / 0.0114
[38/40][44/169] Loss_D: 0.0122 Loss_G: 6.4740 D(x): 0.9958 D(G(z)): 0.0077 / 0.0043
[38/40][45/169] Loss_D: 0.0106 Loss_G: 6.1984 D(x): 0.9974 D(G(z)): 0.0078 / 0.0043
[38/40][46/169] Loss_D: 0.0183 Loss_G: 5.9517 D(x): 0.9991 D(G(z)): 0.0170 / 0.0056
[38/40][47/169] Loss_D: 0.0138 Loss_G: 6.8122 D(x): 0.9897 D(G(z)): 0.0034 / 0.0026
[38/40][48/169] Loss_D: 0.0161 Loss_G: 6.2686 D(x): 0.9879 D(G(z)): 0.0038 / 0.0043
[38/40][49/169] Loss_D: 0.0129 Loss_G: 6.0019 D(x): 0.9955 D(G(z)): 0.0082 / 0.0065
[38/40][50/169] Loss_D: 0.0039 Loss_G: 6.9509 D(x): 0.9982 D(G(z)): 0.0021 / 0.0020
[38/40][51/169] Loss_D: 0.0301 Loss_G: 5.1804 D(x): 0.9756 D(G(z)): 0.0047 /

[38/40][139/169] Loss_D: 0.0264 Loss_G: 6.0410 D(x): 0.9925 D(G(z)): 0.0179 / 0.0049
[38/40][140/169] Loss_D: 0.0179 Loss_G: 7.0613 D(x): 0.9873 D(G(z)): 0.0050 / 0.0039
[38/40][141/169] Loss_D: 0.0052 Loss_G: 7.7238 D(x): 0.9966 D(G(z)): 0.0017 / 0.0016
[38/40][142/169] Loss_D: 0.0057 Loss_G: 7.5095 D(x): 0.9955 D(G(z)): 0.0012 / 0.0013
[38/40][143/169] Loss_D: 0.0047 Loss_G: 7.7614 D(x): 0.9968 D(G(z)): 0.0015 / 0.0016
[38/40][144/169] Loss_D: 0.0185 Loss_G: 5.8059 D(x): 0.9893 D(G(z)): 0.0075 / 0.0071
[38/40][145/169] Loss_D: 0.0127 Loss_G: 5.6361 D(x): 0.9936 D(G(z)): 0.0061 / 0.0061
[38/40][146/169] Loss_D: 0.0084 Loss_G: 6.6563 D(x): 0.9938 D(G(z)): 0.0021 / 0.0026
[38/40][147/169] Loss_D: 0.0163 Loss_G: 5.8781 D(x): 0.9916 D(G(z)): 0.0078 / 0.0070
[38/40][148/169] Loss_D: 0.0137 Loss_G: 6.2184 D(x): 0.9899 D(G(z)): 0.0034 / 0.0041
[38/40][149/169] Loss_D: 0.0681 Loss_G: 3.4759 D(x): 0.9468 D(G(z)): 0.0094 / 0.0494
[38/40][150/169] Loss_D: 0.0680 Loss_G: 5.6958 D(x): 0.9994 D(G(z

[39/40][70/169] Loss_D: 0.0068 Loss_G: 6.5771 D(x): 0.9968 D(G(z)): 0.0035 / 0.0028
[39/40][71/169] Loss_D: 0.0105 Loss_G: 7.1510 D(x): 0.9910 D(G(z)): 0.0013 / 0.0018
[39/40][72/169] Loss_D: 0.0123 Loss_G: 5.9736 D(x): 0.9947 D(G(z)): 0.0069 / 0.0057
[39/40][73/169] Loss_D: 0.0050 Loss_G: 6.6840 D(x): 0.9982 D(G(z)): 0.0031 / 0.0027
[39/40][74/169] Loss_D: 0.0092 Loss_G: 7.6658 D(x): 0.9916 D(G(z)): 0.0007 / 0.0010
[39/40][75/169] Loss_D: 0.0139 Loss_G: 6.1963 D(x): 0.9983 D(G(z)): 0.0119 / 0.0048
[39/40][76/169] Loss_D: 0.0205 Loss_G: 5.6438 D(x): 0.9852 D(G(z)): 0.0053 / 0.0061
[39/40][77/169] Loss_D: 0.0116 Loss_G: 5.9662 D(x): 0.9994 D(G(z)): 0.0108 / 0.0054
[39/40][78/169] Loss_D: 0.0060 Loss_G: 8.2339 D(x): 0.9948 D(G(z)): 0.0008 / 0.0008
[39/40][79/169] Loss_D: 0.0223 Loss_G: 5.9742 D(x): 0.9928 D(G(z)): 0.0145 / 0.0063
[39/40][80/169] Loss_D: 0.0030 Loss_G: 7.6307 D(x): 0.9983 D(G(z)): 0.0014 / 0.0011
[39/40][81/169] Loss_D: 0.0192 Loss_G: 5.8787 D(x): 0.9878 D(G(z)): 0.0067 /

[39/40][167/169] Loss_D: 0.2786 Loss_G: 2.7197 D(x): 0.8672 D(G(z)): 0.0963 / 0.1252
[39/40][168/169] Loss_D: 0.3429 Loss_G: 3.3370 D(x): 0.9174 D(G(z)): 0.1752 / 0.0514


In [59]:
# do checkpointing
torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))
torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outf, epoch))